In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

In [2]:
enade_df = pd.read_csv('enade_merge_08-11-14-17.csv',sep=',',decimal='.')

In [3]:
enade_df

,ano,curso,nome_curso,uf,regiao,codigo_IES,categoria_adm_IES,organizacao_academica,turno,ead_presencial,...,moradia,pessoas_moradia,ensino_medio,condicoes_salas_estudantes,condicoes_praticas_materiais,plano_ensino,avaliacao_curso,idade_comeco_graduacao,tempo_ocioso,tempo_cursado
0,2008,56172,SI,BA,NE,385,IES privada,Universidade,Noturno,Não-Informado,...,Parentes,Uma,Outro,Todas,Todas,Nenhum,Boa,18,1,6
1,2008,8913,SI,RS,S,426,IES privada,Centro Universitário,Noturno,Não-Informado,...,Parentes,Uma,Outro,Maior parte,Maior parte,Maior parte,Boa,20,4,4
2,2008,10978,SI,RS,S,532,IES privada,Universidade,Noturno,Não-Informado,...,Outras pessoas,Nenhuma,Outro,Maior parte,Maior parte,Maior parte,Regular,25,6,2
3,2008,20469,SI,PR,S,197,IES privada,Faculdade,Noturno,Não-Informado,...,Parentes,Nenhuma,Outro,Nenhuma,Nenhuma,Nenhum,Muito fraca,21,5,3
4,2008,47985,ADS,SP,SE,1743,IES privada,Faculdade,Noturno,Não-Informado,...,Parentes,Uma,Outro,Algumas,Algumas,Somente alguns,Muito boa,18,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132577,2017,5001211,ADS,SC,S,3295,IES privada,Faculdade,Diurno,Presencial,...,Parentes,Quatro,EM tradicional,Todas,Todas,Maior parte,Muito boa,18,1,2
132578,2017,5001211,ADS,SC,S,3295,IES privada,Faculdade,Diurno,Presencial,...,Sozinho,Nenhuma,EM tradicional,Maior parte,Maior parte,Somente alguns,Regular,23,5,2
132579,2017,5001211,ADS,SC,S,3295,IES privada,Faculdade,Diurno,Presencial,...,Parentes,Três,EM tradicional,Todas,Todas,Somente alguns,Regular,34,9,1
132580,2017,5001284,LCC,SC,S,494,IES privada,Universidade,Integral,EAD,...,Parentes,Três,EM tradicional,Nenhuma,Todas,Todos aspectos,Muito boa,23,6,2


### Colunas removidas

Algumas colunas foram removidas, pois se utilizadas gerariam muitas outras colunas.
As colunas removidas foram
- curso
- codigo_IES
- formatura
- inicio graduacao
- idade

In [4]:
colunas_categoricas = ['ano', 'nome_curso','uf', 'regiao', 'categoria_adm_IES', 'organizacao_academica', 'turno', 'ead_presencial', 
                       'tipo_ensino_medio', 'range_idade', 'sexo', 'cor_raca', 'escolaridade_pai', 'escolaridade_mae', 
                       'renda_familiar', 'cotas', 'recebeu_cota', 'bolsa_academica', 'recebeu_bolsa', 'trabalha', 'livros_ano',
                       'horas_estudo', 'estado_civil', 'moradia', 'pessoas_moradia', 'ensino_medio', 'condicoes_salas_estudantes',
                       'condicoes_praticas_materiais', 'plano_ensino', 'avaliacao_curso', 'idade_comeco_graduacao', 'tempo_ocioso', 'tempo_cursado']

In [5]:
copia_df = enade_df.copy()

In [6]:
# deixar o dataset apenas com colunas que podem passar por one-hot encode

colunas_df = list(enade_df.columns)
for i in colunas_df:
    if i not in colunas_categoricas:
        enade_df.drop(columns=i, inplace=True)

### Label encoder
transformar categorias em variavel númérica

In [7]:
translate_df = {}

In [8]:
label_encoder = LabelEncoder()

for i in colunas_categoricas:
    enade_df[i] = label_encoder.fit_transform(enade_df[i])
    translate_df[i] = list(label_encoder.classes_)

In [9]:
enade_df

,ano,nome_curso,uf,regiao,categoria_adm_IES,organizacao_academica,turno,ead_presencial,tipo_ensino_medio,range_idade,...,moradia,pessoas_moradia,ensino_medio,condicoes_salas_estudantes,condicoes_praticas_materiais,plano_ensino,avaliacao_curso,idade_comeco_graduacao,tempo_ocioso,tempo_cursado
0,0,6,4,2,0,5,2,1,1,0,...,5,9,3,6,6,1,0,4,1,6
1,0,6,22,3,0,1,2,1,1,0,...,5,9,3,1,1,0,0,6,4,4
2,0,6,22,3,0,5,2,1,1,1,...,3,3,3,1,1,0,7,11,6,2
3,0,6,17,3,0,2,2,1,1,0,...,5,3,3,2,2,1,3,7,5,3
4,0,0,25,4,0,2,2,1,1,0,...,5,9,3,0,0,5,2,4,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132577,3,0,23,3,0,2,0,2,3,0,...,5,6,1,6,6,0,2,4,1,2
132578,3,0,23,3,0,2,0,2,2,0,...,6,3,1,1,1,5,7,9,5,2
132579,3,0,23,3,0,2,0,2,3,2,...,5,8,1,6,6,5,7,20,9,1
132580,3,4,23,3,0,5,1,0,3,0,...,5,8,1,2,6,6,2,9,6,2


In [10]:
enade_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132582 entries, 0 to 132581
Data columns (total 33 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   ano                           132582 non-null  int64
 1   nome_curso                    132582 non-null  int64
 2   uf                            132582 non-null  int64
 3   regiao                        132582 non-null  int64
 4   categoria_adm_IES             132582 non-null  int64
 5   organizacao_academica         132582 non-null  int64
 6   turno                         132582 non-null  int64
 7   ead_presencial                132582 non-null  int64
 8   tipo_ensino_medio             132582 non-null  int64
 9   range_idade                   132582 non-null  int64
 10  sexo                          132582 non-null  int64
 11  cor_raca                      132582 non-null  int64
 12  escolaridade_pai              132582 non-null  int64
 13  escolaridade_m

In [11]:
enade_df.describe()

,ano,nome_curso,uf,regiao,categoria_adm_IES,organizacao_academica,turno,ead_presencial,tipo_ensino_medio,range_idade,...,moradia,pessoas_moradia,ensino_medio,condicoes_salas_estudantes,condicoes_praticas_materiais,plano_ensino,avaliacao_curso,idade_comeco_graduacao,tempo_ocioso,tempo_cursado
count,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,...,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000,132582.000000
mean,1.773446,2.931318,17.232128,3.003289,0.277677,3.271100,1.605557,1.720007,2.587915,0.643911,...,3.850689,5.275256,1.749838,3.353600,3.123403,3.177490,2.527621,8.739437,4.526067,3.552941
std,1.029036,2.540376,7.391493,1.286324,0.447855,1.665961,0.714375,0.585075,0.591502,0.875295,...,2.067283,3.220855,1.537772,2.551928,2.542716,2.798074,2.575301,6.020675,4.952888,1.843693
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,10.000000,2.000000,0.000000,2.000000,1.000000,2.000000,2.000000,0.000000,...,3.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,5.000000,1.000000,2.000000
50%,2.000000,2.000000,18.000000,4.000000,0.000000,3.000000,2.000000,2.000000,3.000000,0.000000,...,5.000000,6.000000,1.000000,3.000000,2.000000,5.000000,2.000000,7.000000,3.000000,3.000000
75%,3.000000,6.000000,25.000000,4.000000,1.000000,5.000000,2.000000,2.000000,3.000000,1.000000,...,5.000000,8.000000,1.000000,6.000000,6.000000,6.000000,3.000000,11.000000,6.000000,4.000000
max,3.000000,6.000000,26.000000,4.000000,1.000000,5.000000,3.000000,2.000000,3.000000,4.000000,...,7.000000,9.000000,5.000000,6.000000,6.000000,6.000000,7.000000,54.000000,46.000000,23.000000


### One Hot Encoding
transformar variaveis núméricas em colunas de valores binarios

https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/

https://arthurlambletvaz.medium.com/one-hot-encoding-o-que-%C3%A9-cd2e8d302ae0

https://minerandodados.com.br/one-hot-encoding-como-funciona-python/

https://stackify.dev/928474-scikit-learns-labelbinarizer-vs-onehotencoder

https://stackoverflow.com/questions/58101126/using-scikit-learn-onehotencoder-with-a-pandas-dataframe#:~:text=OneHotEncoder%20Encodes%20categorical%20integer%20features,matrix)%20into%20a%20Pandas%20Series.

In [12]:
onehot_encoder = OneHotEncoder(sparse=False)

In [13]:
newdf = pd.DataFrame()

In [14]:
encoder = LabelBinarizer()

for i in colunas_categoricas:
    encoder.fit(enade_df[i])
    transformed = encoder.transform(enade_df[i])
    ohe_df = pd.DataFrame(transformed)
    # ohe_df.rename(columns=lambda x: i+'_'+str(x) , inplace=True)
    ohe_df.rename(columns=lambda x: i+'_'+ str(translate_df[i][x]) , inplace=True)
    newdf = pd.concat([newdf, ohe_df], axis=1)

In [15]:
newdf

,ano_2008,ano_2011,ano_2014,ano_2017,nome_curso_ADS,nome_curso_BCC,nome_curso_EC,nome_curso_GTI,nome_curso_LCC,nome_curso_RC,...,tempo_cursado_14,tempo_cursado_15,tempo_cursado_16,tempo_cursado_17,tempo_cursado_18,tempo_cursado_19,tempo_cursado_20,tempo_cursado_21,tempo_cursado_23,tempo_cursado_24
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132577,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132578,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132579,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132580,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
copia_df = copia_df[['nota_geral', 'nota_formacao_geral', 'nota_obj_formacao_geral', 'nota_dis_formacao_geral', 'nota_componente_especifico', 'nota_obj_componente_especifico',
                    'nota_dis_componente_especifico']]

In [17]:
newdf = pd.concat([newdf, copia_df], axis=1)

In [18]:
newdf

,ano_2008,ano_2011,ano_2014,ano_2017,nome_curso_ADS,nome_curso_BCC,nome_curso_EC,nome_curso_GTI,nome_curso_LCC,nome_curso_RC,...,tempo_cursado_21,tempo_cursado_23,tempo_cursado_24,nota_geral,nota_formacao_geral,nota_obj_formacao_geral,nota_dis_formacao_geral,nota_componente_especifico,nota_obj_componente_especifico,nota_dis_componente_especifico
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,19.7,7.5,12.5,0.0,23.8,28.0,0.0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,46.3,37.5,62.5,0.0,49.2,52.0,33.3
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,40.3,67.0,75.0,55.0,31.4,36.0,5.6
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,20.5,51.5,62.5,35.0,10.2,12.0,0.0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,31.4,27.0,25.0,30.0,32.9,36.8,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132577,0,0,0,1,1,0,0,0,0,0,...,0,0,0,46.8,37.9,37.5,38.5,49.8,55.0,20.0
132578,0,0,0,1,1,0,0,0,0,0,...,0,0,0,26.9,49.5,37.5,67.5,19.3,20.0,15.0
132579,0,0,0,1,1,0,0,0,0,0,...,0,0,0,20.5,18.0,25.0,7.5,21.3,25.0,0.0
132580,0,0,0,1,0,0,0,0,1,0,...,0,0,0,52.0,54.8,50.0,62.0,51.0,56.5,20.0


In [19]:
newdf.to_csv('./{}.csv'.format('enade_onehot'),index=False,float_format='%.2f')